In [1]:
# week1_stock_analysis_50.py

import matplotlib
matplotlib.use("Agg")  # Non-interactive backend

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import time
from datetime import datetime

warnings.simplefilter(action='ignore', category=FutureWarning)

# List of 50 stock symbols
stocks = [
    "AAPL","MSFT","GOOGL","AMZN","TSLA","META","NVDA","JPM","V","UNH",
    "HD","PG","DIS","MA","BAC","NFLX","ADBE","PYPL","CMCSA","XOM",
    "PFE","KO","INTC","CSCO","NKE","MRK","PEP","ABBV","ABT","CRM",
    "ORCL","T","VZ","CVX","MCD","WMT","DHR","ACN","LLY","AVGO",
    "QCOM","TXN","NEE","COST","TMUS","IBM","SBUX","MDT","HON","AMD"
]

# Create necessary folders
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)
os.makedirs("../plots/ratio", exist_ok=True)
os.makedirs("../plots/correlation", exist_ok=True)

processed_stocks = []
skipped_stocks = []

# Timestamp for unique file names
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for symbol in stocks:
    print(f"\n--- Processing {symbol} ---")

    raw_file = f"../data/raw/{symbol}_raw.csv"
    processed_file = f"../data/processed/{symbol}_processed_{timestamp}.csv"
    plot_file = f"../plots/ratio/{symbol}_price_trend_{timestamp}.png"

    # 1. Load or download data
    if os.path.exists(raw_file):
        print(f"{symbol} raw data already exists. Loading from file.")
        data = pd.read_csv(raw_file, index_col=0, parse_dates=True)
    else:
        try:
            data = yf.download(symbol, start="2015-01-01", end="2025-01-01")
            if data.empty:
                print(f"Warning: {symbol} data is empty. Skipping.")
                skipped_stocks.append(symbol)
                continue
            data.to_csv(raw_file)
            time.sleep(1)  # Delay to avoid Yahoo blocking
        except Exception as e:
            print(f"Error downloading {symbol}: {e}")
            skipped_stocks.append(symbol)
            continue

    # Flatten multi-level columns if present
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    # Convert numeric columns safely
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    for col in numeric_cols:
        if col in data.columns and isinstance(data[col], pd.Series):
            data[col] = pd.to_numeric(data[col], errors='coerce')

    # Use Adj Close if Close is missing
    if 'Close' not in data.columns and 'Adj Close' in data.columns:
        data['Close'] = data['Adj Close']

    if 'Close' not in data.columns or data['Close'].isna().all():
        print(f"Warning: {symbol} has no valid Close data. Skipping.")
        skipped_stocks.append(symbol)
        continue


    # 1. Drop rows with missing Close
    data.dropna(subset=['Close'], inplace=True)

    # 2. Forward-fill other numeric columns
    for col in ['Open', 'High', 'Low', 'Adj Close', 'Volume']:
        if col in data.columns and isinstance(data[col], pd.Series):
            data[col].fillna(method='ffill', inplace=True)

    # 3. Remove duplicate dates
    data = data[~data.index.duplicated(keep='first')]

    # 4. Sort by date
    data.sort_index(inplace=True)

    # Feature Engineering
    data['Return'] = data['Close'].pct_change()
    data['Next_Return'] = data['Close'].pct_change().shift(-1) * 100
    data['Daily_Return'] = data['Close'].pct_change() * 100
    data['Return_lag1'] = data['Daily_Return'].shift(1)
    data['Return_lag2'] = data['Daily_Return'].shift(2)
    data['Return_lag3'] = data['Daily_Return'].shift(3)

    data['Volatility'] = data['Return'].rolling(20).std()

    data['High_Low_Ratio'] = data['High'] / data['Low']
    data['Open_Close_Ratio'] = data['Open'] / data['Close']
    data['High_Low_Range'] = (data['High'] - data['Low']) / data['Close'] * 100
    data['Close_to_Open'] = (data['Close'] - data['Open']) / data['Open'] * 100

    # Volume changes (% change, not absolute)
    data['Volume_Change'] = data['Volume'].pct_change() * 100
    data['Volume_lag1'] = data['Volume_Change'].shift(1)


    data['MA_5'] = data['Close'].rolling(5).mean()
    data['MA_10'] = data['Close'].rolling(10).mean()
    data['MA_20'] = data['Close'].rolling(20).mean()
    data['MA_30'] = data['Close'].rolling(30).mean()
    data['Distance_from_MA_5'] = (data['Close'] - data['MA_5']) / data['MA_5'] * 100
    data['Distance_from_MA_10'] = (data['Close'] - data['MA_10']) / data['MA_10'] * 100
    data['Distance_from_MA_20'] = (data['Close'] - data['MA_20']) / data['MA_20'] * 100
    data['Distance_from_MA_30'] = (data['Close'] - data['MA_30']) / data['MA_30'] * 100


    # Save processed data
    data.to_csv(processed_file)

    # Plot closing price and MA
    plt.figure(figsize=(12,6))
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['MA_5'], label='MA_5', linestyle='--')
    plt.plot(data['Volatility'], label='Volatility', linestyle='--')
    plt.title(f"{symbol} Stock Price Trend")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.savefig(plot_file)
    plt.close()

    feature_cols = [col for col in data.columns if col not in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

    # Correlation matrix
    corr = data[feature_cols].corr()

    # Plot heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
    plt.title(f"{symbol} Feature Correlation")
    plt.tight_layout()
    plt.savefig(f"../plots/correlation/{symbol}_correlation.png")
    plt.close()
    
    processed_stocks.append(symbol)

print("\nWeek 1: Stocks processed successfully.")
print("Processed stocks:", processed_stocks)
print("Skipped stocks:", skipped_stocks)



--- Processing AAPL ---
AAPL raw data already exists. Loading from file.


C:\Users\Mahdin\AppData\Local\Temp\ipykernel_4032\3850144567.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(raw_file, index_col=0, parse_dates=True)



--- Processing MSFT ---
MSFT raw data already exists. Loading from file.


C:\Users\Mahdin\AppData\Local\Temp\ipykernel_4032\3850144567.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(raw_file, index_col=0, parse_dates=True)


KeyboardInterrupt: 